# Sparse Grid Regression with the Combination Technique: 


### Utility functions :

In [ ]:
from sparseSpACE.Function import *
from sparseSpACE.Utils import *
from sparseSpACE.DEMachineLearning import *
import numpy as np
import math

def construct_dataset(dim, function, num_points):
    grid_points = np.random.rand(num_points, dim)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

    
#def split_dataset(data, targets, parts):
#    training_size = 0
#    test_size = 0
#    for i in range(len(data)):
#        if i%parts != 0:
#            training_size += 1
#        else:
#            test_size += 1
#            
#    training_data = np.zeros((training_size, len(data[0]))) 
#    training_targets = np.zeros(training_size) 
#    
#    test_data = np.zeros((test_size, len(data[0]))) 
#    test_targets = np.zeros(test_size) 
#    
#    training_index = 0
#    test_index = 0
#    
#    for i in range(len(data)):
#        if i%parts != 0:
#            training_data[training_index] = data[i]
#            training_targets[training_index] = targets[i]
#            training_index += 1
#       else:
#            test_data[test_index] = data[i]
#            test_targets[test_index] = targets[i]
#            test_index += 1
#    
#    return training_data, training_targets, test_data, test_targets


#def train_regression(training_data, training_targets, regularization, matrix, minimum_level, maximum_level):
#    dim = len(training_data[0])
#    
#    a = np.zeros(dim)
#    b = np.ones(dim)
#    
#    operation = Regression(training_data, training_targets, regularization, regularization_matrix=matrix)
#    
#    combiObject = StandardCombi(a, b, operation=operation)
#    
#    combiObject.perform_operation(minimum_level, maximum_level)
#    
#    return operation, combiObject


#def test_regression(test_data, test_targets, combiObject):
#    learned_targets = combiObject(test_data)
#    
#    difference = 0
#    for i in range(len(learned_targets)):
#        difference += (test_targets[i] - learned_targets[i]) ** 2
#        
#    return math.sqrt(difference/len(test_targets))

#def scale_data(data, target, rangee=[0.05,0.95]):
#    dataSet = DataSetRegression((data, target))
#    dataSet.scale_range(rangee)
#    data, target = dataSet.get_data()[0], dataSet.get_data()[1]
#    return data, target


def test_regression_spatially_adaptive(test_data, test_targets, adaptiveCombiInstanceSingleDim, grid):
    learned_targets = adaptiveCombiInstanceSingleDim(test_data)
    
    difference = 0
    for i in range(len(learned_targets)):
        difference += (test_targets[i] - learned_targets[i]) ** 2
        
    return math.sqrt(difference/len(test_targets))

def find_best_lambda(data, targets, matrix, level_min=1, level_max=5):
    lambdas = np.zeros(7)
    
    for i in range(3,10):
        operation = Regression(data, target, 10**-i, matrix)
        combiObject = operation.train(0.4, 1, 5)
        lambdas[i-3] = operation.test(combiObject)
        
    exp = np.where(lambdas == np.amin(lambdas))
    return 10.**-(exp[0][0])

## Test: Increasing size of dataset -> Error


In [ ]:
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))

stringBuilderC = ""
for i in range(1,11):
    data, target = construct_dataset(2, func, (i*10)**2)
    lambdaC = find_best_lambda(data, target, 'C', 1, 5)
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderC += "" + str((i*10)**2) + " & "+ str(operation.test(combiObject)) + "\\\\\n"

stringBuilderI = ""
for i in range(1,11):
    data, target = construct_dataset(2, func, (i*10)**2)
    lambdaI = find_best_lambda(data, target, 'I', 1, 5)
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderI += "" + str((i*10)**2) + " & "+ str(operation.test(combiObject)) + "\\\\\n"
    
print("Testfehler:")
print(stringBuilderC)
print(stringBuilderI)

## Test: Increasing number of grid points -> Error


In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 400)


stringBuilderC = ""
for i in range(2, 9):
    lambdaC = find_best_lambda(data, target, 'C', 1, i)
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(0.2, 1, i)
    stringBuilderC += ""+str(operation.get_grid().get_num_points())+" & "+str(operation.test(combiObject))+"\\\\\n"

    
stringBuilderI = ""
for i in range(2, 9):
    lambdaI = find_best_lambda(data, target, 'I', 1, i)
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(0.2, 1, i)
    stringBuilderI += ""+str(operation.get_grid().get_num_points())+" & "+str(operation.test(combiObject))+"\\\\\n"
  
    
print("Testfehler:")
print(stringBuilderC)
print(stringBuilderI)

## Test: Increasing regularization parameter lambda -> Error


In [ ]:
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
from sklearn import datasets

data, target = datasets.load_diabetes(return_X_y=True)

stringBuilderC = ""
for i in range(10):
    operation = Regression(data, target, 10**-i, 'C')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderC += ""+str(10**-i)+" & " + str(operation.test(combiObject))+"\\\\\n"

stringBuilderI = ""
for i in range(10):
    operation = Regression(data, target, 10**-i, 'I')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderI += ""+str(10**-i)+" & " + str(operation.test(combiObject))+"\\\\\n"
    
print("Tabelle:")
print(stringBuilderC)
print(stringBuilderI)


## Test: Increasing regularization parameter lambda -> Error


In [ ]:
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
from sklearn import datasets

data, target = datasets.load_boston(return_X_y=True)

stringBuilderC = ""
for i in range(10):
    operation = Regression(data, target, 10**-i, 'C')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderC += ""+str(10**-i)+" & " + str(operation.test(combiObject))+"\\\\\n"

stringBuilderI = ""
for i in range(10):
    operation = Regression(data, target, 10**-i, 'I')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderI += ""+str(10**-i)+" & " + str(operation.test(combiObject))+"\\\\\n"
    
print("Tabelle:")
print(stringBuilderC)
print(stringBuilderI)


## Test: Increase test percentage -> Error


In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 400)

lambdaC = find_best_lambda(data, target, 'C', 1, 5)
lambdaI = find_best_lambda(data, target, 'I', 1, 5)

stringBuilderC = ""
for i in range(1, 10):
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(i/10, 1, 5)
    stringBuilderC += ""+str(i/10)+" & " + str(operation.test(combiObject))+"\\\\\n"
    
stringBuilderI = ""
for i in range(1, 10):
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(i/10, 1, 5)
    stringBuilderI += ""+str(i/10)+" & " + str(operation.test(combiObject))+"\\\\\n"

print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)

## Test: Increasing size of data set -> Time


In [ ]:
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))

stringBuilderC = ""
for i in range(1,11):
    data, target = construct_dataset(2, func, (i*10)**2)
    lambdaC = find_best_lambda(data, target, 'C', 1, 5)
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(0.1, 1, 5)
    stringBuilderC += "" + str((i*10)**2) + " & "+ str(combiObject.get_time_used()) + "\\\\\n"

stringBuilderI = ""
for i in range(1,11):
    data, target = construct_dataset(2, func, (i*10)**2)
    lambdaI = find_best_lambda(data, target, 'I', 1, 5)
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(0.1, 1, 5)
    stringBuilderI += "" + str((i*10)**2) + " & "+ str(combiObject.get_time_used()) + "\\\\\n"
    
print("Testfehler:")
print(stringBuilderC)
print(stringBuilderI)

## Test: Increasing number of grid points -> Time


In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 400)


stringBuilderC = ""
for i in range(2, 9):
    lambdaC = find_best_lambda(data, target, 'C', 1, i)
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(0.1, 1, i)
    stringBuilderC += ""+str(operation.get_grid().get_num_points())+" & "+str(combiObject.get_time_used())+"\\\\\n"

    
stringBuilderI = ""
for i in range(2, 9):
    lambdaI = find_best_lambda(data, target, 'I', 1, i)
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(0.1, 1, i)
    stringBuilderI += ""+str(operation.get_grid().get_num_points())+" & "+str(combiObject.get_time_used())+"\\\\\n"
  
    
print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)

## Test: Increase dimension -> Error

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (5,5))
func.plot((0,0),(1,1))


stringBuilderC = ""
for i in range(2, 5):
    liste = [0.5 for _ in range(i)]
    liste2 = [5 for _ in range(i)]
    func = GenzGaussian(tuple(liste), tuple(liste2))
    data, target = construct_dataset(i, func, 400)
    lambdaC = find_best_lambda(data, target, 'C', 1, 5)
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderC += ""+str(i)+" & "+str(operation.test(combiObject))+"\\\\\n"

    
stringBuilderI = ""
for i in range(2, 5):
    liste = [0.5 for _ in range(i)]
    liste2 = [5 for _ in range(i)]
    func = GenzGaussian(tuple(liste), tuple(liste2))
    data, target = construct_dataset(i, func, 400)
    lambdaI = find_best_lambda(data, target, 'I', 1, 5)
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderI += ""+str(i)+" & "+str(operation.test(combiObject))+"\\\\\n"
  
    
print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)

## Test: Increase dimension -> Time

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (5,5))
func.plot((0,0),(1,1))


stringBuilderC = ""
for i in range(2, 10):
    liste = [0.5 for _ in range(i)]
    liste2 = [5 for _ in range(i)]
    func = GenzGaussian(tuple(liste), tuple(liste2))
    data, target = construct_dataset(i, func, 400)
    lambdaC = find_best_lambda(data, target, 'C', 1, 5)
    operation = Regression(data, target, lambdaC, 'C')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderC += ""+str(i)+" & "+str(combiObject.get_time_used())+"\\\\\n"

    
stringBuilderI = ""
for i in range(2, 10):
    liste = [0.5 for _ in range(i)]
    liste2 = [5 for _ in range(i)]
    func = GenzGaussian(tuple(liste), tuple(liste2))
    data, target = construct_dataset(i, func, 400)
    lambdaI = find_best_lambda(data, target, 'I', 1, 5)
    operation = Regression(data, target, lambdaI, 'I')
    combiObject = operation.train(0.2, 1, 5)
    stringBuilderI += ""+str(i)+" & "+str(combiObject.get_time_used())+"\\\\\n"
  
    
print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)

# Spatially adaptive

## Test: Increase margin

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 400)


stringBuilderC = ""
for i in range(0, 11):
    operation = Regression(data, target, 10**-6, 'C')
    adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=i/10, tolerance=10**-3, max_evaluations=500)
    error = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
    stringBuilderC += ""+str(i/10)+" & "+str(error)+"\\\\\n"

stringBuilderI = ""
for i in range(0, 11):
    operation = Regression(data, target, 10**-6, 'I')
    adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=i/10, tolerance=10**-3, max_evaluations=500)
    error = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
    stringBuilderI += ""+str(i/10)+" & "+str(error)+"\\\\\n"

    
print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)

## Test: Increasing lambda -> Fault


In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 400)


stringBuilderC = ""
for i in range(0, 11):
    operation = Regression(data, target, 10**-i, 'C')
    adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=0.4, tolerance=10**-3, max_evaluations=500)
    error = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
    stringBuilderC += ""+str(10**-i)+" & "+str(error)+"\\\\\n"

stringBuilderI = ""
for i in range(0, 11):
    operation = Regression(data, target, 10**-i, 'I')
    adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=0.4, tolerance=10**-3, max_evaluations=500)
    error = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
    stringBuilderI += ""+str(10**-i)+" & "+str(error)+"\\\\\n"

    
print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)

## Test: Increasing max evaluations -> Fault


In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 400)


stringBuilderC = ""
for i in range(1, 5):
    operation = Regression(data, target, 10**-6, 'C')
    adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=0.4, tolerance=10**-3, max_evaluations=(10*i)**2)
    error = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
    stringBuilderC += ""+str((10*i)**2)+" & "+str(error)+"\\\\\n"

stringBuilderI = ""
for i in range(1, 5):
    operation = Regression(data, target, 10**-6, 'I')
    adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=0.4, tolerance=10**-3, max_evaluations=(10*i)**2)
    error = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
    stringBuilderI += ""+str((10*i)**2)+" & "+str(error)+"\\\\\n"

    
print("Matrix C:")
print(stringBuilderC)
print("Matrix I:")
print(stringBuilderI)